# Entendimiento del Negocio - Contexto - Entrega 4

## 3.2 Carga de los datos




# Ficha técnica del despliegue del modelo

## Nombre del modelo: **xgboost_credit_risk**

Es el nombre registrado en el MLflow Model Registry para el modelo optimizado con XGBoost que predice la probabilidad de incumplimiento de pago de clientes de tarjetas de crédito.



## Plataforma de despliegue
MLflow Models + REST API local (localhost:8001) expuesta vía ngrok

El modelo es desplegado usando mlflow models serve, que lanza un servidor REST local. Luego, ngrok permite exponer ese puerto a Internet de manera segura.
Este enfoque simula un entorno de producción básico, ideal para demostraciones, pruebas y desarrollo ágil.

## Requisitos técnicos


Requisito	Versión / Especificación
Python	3.10 o superior
MLflow	≥ 2.11.1
XGBoost	≥ 2.0.3
scikit-learn	≥ 1.3.0
Pandas	≥ 2.0
PyArrow	≥ 14.0.0 (para compatibilidad con dtype_backend="pyarrow")
requests	≥ 2.31.0
pyngrok	≥ 6.0.0
Hardware mínimo	CPU de 2 núcleos, 4GB RAM (para despliegue local básico)
Sistema operativo	Windows, Linux o Google Colab (simulado)

## Requisitos de seguridad

Como es un entorno de desarrollo, se plantean requisitos mínimos. Para producción se requerirían mecanismos más robustos.

Requisito	Estado / Implementación sugerida
Autenticación	No implementada en el ejemplo (se recomienda JWT o API Key en producción)
Encriptación en tránsito	ngrok crea túneles HTTPS seguros
Aislamiento del entorno	Entorno virtual recomendado (venv, conda, docker)
Acceso restringido	Solo exponer ngrok durante pruebas
Registro de logs	MLflow registra métricas y artefactos

## Seguridad del despliegue (visión de estudiante)

### ¿Qué tan seguro es **lo que hicimos**?

| Componente | ¿Qué hicimos? | Riesgos | Medidas mínimas que aplicamos |
|------------|---------------|---------|-------------------------------|
| **MLflow Tracking** | Carpeta local `./mlruns` con todos los runs y el Model Registry | • Los artefactos no están cifrados.<br>• Cualquiera con acceso al PC puede modificar o borrar runs. | • Carpeta dentro del proyecto (no en ruta pública).<br>• Permisos de SO heredados: solo nuestra cuenta de usuario la ve. |
| **MLflow Serve (Uvicorn)** | Servidor REST en `http://localhost:5001` | • Sin TLS (HTTP plano).<br>• No hay autenticación. | • Solo se abre en la máquina local. |
| **Ngrok** | Túnel HTTPS → URL pública `https://XXXX.ngrok-free.app` | • Cualquiera con la URL puede llamar al modelo mientras esté activo.<br>• El token de ngrok vive en la variable de entorno del usuario. | • Ngrok usa HTTPS por defecto (cifrado en tránsito).<br>• El túnel se cierra al parar la terminal. |

> **Conclusión:** Para un laboratorio o demo dentro del banco es suficiente,  
> porque el puerto real (5001) nunca sale de la máquina y el túnel ngrok solo vive mientras hacemos la prueba.

---

### ¿Qué faltaría en producción real?

| Requisito               | ¿Por qué es importante?                                          | ¿Cómo podríamos implementarlo?                            |
|-------------------------|------------------------------------------------------------------|------------------------------------------------------------|
| **Control de acceso**   | Garantizar que solo usuarios autorizados usen la API            | Añadir un sistema de claves (API Key) o autenticación básica |
| **Cifrado de datos**    | Proteger la información mientras viaja por internet              | Forzar HTTPS en todas las peticiones                       |
| **Entornos aislados**   | Evitar interferencias y “funciona sólo en mi máquina”           | Ejecutar el servicio en un contenedor (p. ej. Docker)      |
| **Registro de actividad** | Saber quién y cuándo hizo cada petición para auditoría        | Guardar logs de acceso y errores en un archivo o sistema centralizado |
| **Copias de seguridad** | Recuperar el modelo y los datos ante fallos o borrados          | Realizar backups periódicos de la carpeta `mlruns`         |


> **En resumen:**  
> El despliegue actual funciona bien para pruebas internas con ngrok y MLflow, pero para un entorno productivo haríamos:
> - Autenticación para controlar quién usa el servicio.  
> - HTTPS obligatorio para proteger los datos en tránsito.  
> - Aislamiento en contenedores (Docker) para evitar dependencias rotas.  
> - Logs centralizados para auditar uso y errores.  
> - Backups periódicos de los artefactos (`mlruns`) para recuperación ante fallos.

---


## Diagrama de arquitectura del sistema de despliegue

     +----------------------+
     |   KaggleHub Dataset  |
     +----------------------+
                |
                v
     +----------------------+
     | Entrenamiento modelo |
     | (XGBoost + Pipeline) |
     +----------------------+
                |
                v
     +----------------------+
     | Registro en MLflow   |
     | (Tracking + Registry)|
     +----------------------+
                |
                v
     +--------------------------+
     | Despliegue vía MLflow    |
     | REST API (localhost:8001)|
     +--------------------------+
                |
                v
     +----------------------+
     |    pyngrok tunnel    |
     | (https://xxxxx.ngrok.io)|
     +----------------------+
                |
                v
     +----------------------+
     |  Cliente externo/API |
     |   (requests POST)    |
     +----------------------+


## Código de despliegue

| Elemento                           | Descripción                                                                                |
| ---------------------------------- | ------------------------------------------------------------------------------------------ |
| **Archivo principal**              | `deployment_code.py` (contiene entrenamiento, registro y utilidades para servir el modelo) |
| **Rutas de acceso a los archivos** |                                                                                            |
|   • Dataset                        | `./data/UCI_Credit_Card.csv`                                                               |
|   • Carpeta de experimentos (runs) | `./mlruns/0/<run_id>/artifacts/model/`                                                     |
|   • Model Registry                 | `./mlruns/models/XGBoost_Optimizado/`                                                      |
|   • Payload de prueba              | `./payload.json`                                                                           |
| **Variables de entorno**           |                                                                                            |
|   • `MLFLOW_TRACKING_URI`          | `file:///…/scripts/deployment/mlruns`                                                      |
|   • `PORT`                         | `5001` (puerto del servidor de inferencia)                                                 |
|   • `NGROK_AUTHTOKEN`              | * token Ngrok*                                                               |

---

### Pasos para reproducir el despliegue

1. **Entrenar y registrar el modelo**


   Esto guarda el run y crea la versión `XGBoost_Optimizado/2` en el Model Registry.

2. **Servir el modelo localmente**

   ```bash
   mlflow models serve \
     --model-uri "models:/XGBoost_Optimizado/2" \
     --host 0.0.0.0 --port 5001 --workers 2 --no-conda
   ```

   El endpoint queda en `http://localhost:5001/invocations`.

3. **Probar la inferencia local**

   ```bash
   curl -X POST http://localhost:5001/invocations \
        -H "Content-Type: application/json" \
        -d @payload.json
   ```

4. **Exponer con Ngrok**

   ```bash
   ngrok authtoken <TU_TOKEN>
   ngrok http 5001
   # Salida típica:
   # Forwarding https://abcd1234.ngrok-free.app -> http://localhost:5001
   ```

5. **Inferencia remota (cualquier PC)**

   ```bash
   curl -X POST https://abcd1234.ngrok-free.app/invocations \
        -H "Content-Type: application/json" \
        -d @payload.json
   ```

   Devuelve, por ejemplo: `{ "predictions": [0] }`.

---

### Conceptos usados

* **Endpoint**: URL que expone una funcionalidad (aquí: `/invocations`).
* **Inferencia**: proceso de generar una predicción con el modelo.
* **Ngrok**: túnel HTTPS temporal que hace pública una URL interna.
* **Model Registry**: módulo de MLflow que versiona y gestiona modelos listos para producción.

Con estos pasos tu compañero puede reproducir el flujo completo y explicar el despliegue en el taller.


# Documentación de Despliegue del Modelo

**Modelo:** Predicción de Incumplimiento de Tarjeta de Crédito (XGBoost + MLflow)

---

## 1. Instrucciones de instalación

| Paso                    | Comando / Acción                                                    | Comentario                                           |
|-------------------------|---------------------------------------------------------------------|------------------------------------------------------|
| Clonar el proyecto      | `git clone https://github.com/<tu-repo>.git`                        | O descargar el ZIP                                   |
| Crear entorno           | `conda create -n creditenv python=3.10 -y`  
`conda activate creditenv` |                                                      |
| Instalar dependencias   | `pip install -r requirements.txt`                                  | Incluye `mlflow`, `xgboost`, `scikit-learn`, etc.     |
| Verificar datos         | `./data/UCI_Credit_Card.csv` debe existir en la ruta indicada      |                                                      |

> **Requisitos mínimos:** Python ≥ 3.8, Conda (o venv) y puerto 5001 libre.

---

## 2. Instrucciones de configuración

1. Definir variables de entorno:
    ```bash
    set MLFLOW_TRACKING_URI=file:///%CD%/mlruns
    set PORT=5001
    ```
2. Entrenar y registrar el modelo (crea versión 2):
    ```bash
    python deployment_code.py
    ```
3. (Opcional) Registrar token de ngrok:
    ```bash
    ngrok authtoken <TU_TOKEN>
    ```

---





## 3. Instrucciones de uso

### 3.1 Servir el modelo localmente

```bash
mlflow models serve \
  --model-uri "models:/XGBoost_Optimizado/2" \
  --host 0.0.0.0 --port 5001 --workers 2 --no-conda
```

El endpoint queda activo en:
`http://localhost:5001/invocations`

### 3.2 Probar la inferencia local

```bash
curl -X POST http://localhost:5001/invocations \
  -H "Content-Type: application/json" \
  -d @payload.json
```

Respuesta esperada:

```json
{"predictions": [0]}
```

### 3.3 Exponer el servicio con ngrok

```bash
grok http 5001
```

Salida típica:

```
Forwarding    https://abcd1234.ngrok-free.app -> http://localhost:5001
```

### 3.4 Inferencia remota (cualquier equipo)

```bash
curl -X POST https://abcd1234.ngrok-free.app/invocations \
  -H "Content-Type: application/json" \
  -d @payload.json
```

---


## 4. Instrucciones de mantenimiento

| Tarea                   | Acción / Comando                                                                                                                                                                                   | Notas                                        |
| ----------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | -------------------------------------------- |
| Detener el servicio     | Presionar `Ctrl + C` en la terminal donde corre `mlflow models serve`                                                                                                                              |                                              |
| Revisar métricas y runs | `mlflow ui --backend-store-uri ./mlruns`<br>Abrir `http://localhost:5000`                                                                                                                          |                                              |
| Actualizar el modelo    | 1. Re-entrenar: `python deployment_code.py`<br>2. Servir nueva versión: `mlflow models serve --model-uri "models:/XGBoost_Optimizado/<versión>" --host 0.0.0.0 --port 5001 --workers 2 --no-conda` | Solo cambia la URI del modelo, no el código  |
| Backup de artefactos    | Copiar la carpeta `./mlruns` o subirla a un almacenamiento externo                                                                                                                                 | Permite recuperar runs y versiones de modelo |
| Rotar token de ngrok    | `ngrok authtoken <NUEVO_TOKEN>`                                                                                                                                                                    |                                              |

---

## 5. Seguridad

| Requisito              | Entorno de desarrollo    | Recomendación para producción                    |
| ---------------------- | ------------------------ | ------------------------------------------------ |
| Control de acceso      | No aplicado              | Implementar autenticación (API Key, JWT)         |
| Cifrado en tránsito    | HTTPS mediante ngrok     | TLS propio con certificados gestionados          |
| Aislamiento de entorno | Entorno Conda            | Contenedor Docker con usuario no-root            |
| Registro de actividad  | Logs locales de MLflow   | Centralizar logs en sistema dedicado (ELK, etc.) |
| Copias de seguridad    | Manual (copiar `mlruns`) | Automatizar backups periódicos                   |

> **Resumen:**
> El despliegue actual es adecuado para pruebas internas, pero en producción sería necesario añadir autenticación, TLS propio, aislamiento en contenedores, logs centralizados y backups automáticos.

---

## 6. Glosario

| Término         | Significado                                                            |
| --------------- | ---------------------------------------------------------------------- |
| Endpoint        | Ruta `/invocations` que recibe datos y devuelve predicciones           |
| Payload         | JSON con formato `dataframe_split` (campos `columns` y `data`)         |
| MLflow Tracking | Carpeta `mlruns` que almacena runs, métricas y artefactos              |
| Model Registry  | Catálogo de versiones (`XGBoost_Optimizado/2`, etc.)                   |
| ngrok           | Herramienta que expone un puerto local a Internet mediante túnel HTTPS |


## DESPLIGUES

### XGBOOST

In [34]:
!pip install mlflow pyngrok requests

In [35]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

file_path = "UCI_Credit_Card.csv"  # nombre exacto

df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "uciml/default-of-credit-card-clients-dataset",
  file_path
)

print("First 5 records:", df.head())


/tmp/ipython-input-35-1678825860.py:6: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


First 5 records:    ID  LIMIT_BAL  SEX  EDUCATION  MARRIAGE  AGE  PAY_0  PAY_2  PAY_3  PAY_4  \
0   1    20000.0    2          2         1   24      2      2     -1     -1   
1   2   120000.0    2          2         2   26     -1      2      0      0   
2   3    90000.0    2          2         2   34      0      0      0      0   
3   4    50000.0    2          2         1   37      0      0      0      0   
4   5    50000.0    1          2         1   57     -1      0     -1      0   

   ...  BILL_AMT4  BILL_AMT5  BILL_AMT6  PAY_AMT1  PAY_AMT2  PAY_AMT3  \
0  ...        0.0        0.0        0.0       0.0     689.0       0.0   
1  ...     3272.0     3455.0     3261.0       0.0    1000.0    1000.0   
2  ...    14331.0    14948.0    15549.0    1518.0    1500.0    1000.0   
3  ...    28314.0    28959.0    29547.0    2000.0    2019.0    1200.0   
4  ...    20940.0    19146.0    19131.0    2000.0   36681.0   10000.0   

   PAY_AMT4  PAY_AMT5  PAY_AMT6  default.payment.next.month  
0      

In [36]:
import kagglehub
import pandas as pd
import pyarrow as pyw

# Download latest version
path = kagglehub.dataset_download("uciml/default-of-credit-card-clients-dataset")

print("Path to dataset files:", path)

# Cargue de la data en un dataframe
df = pd.read_csv(path + "/UCI_Credit_Card.csv",dtype_backend="pyarrow")

Path to dataset files: /kaggle/input/default-of-credit-card-clients-dataset


In [37]:
df.columns

Index(['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0',
       'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'default.payment.next.month'],
      dtype='object')

In [38]:
df["MARRIAGE"].value_counts()

,count
MARRIAGE,
2,15964
1,13659
3,323
0,54


In [39]:
# ------------- 2. Limpieza & transformación -------
# a) Target – renombra por comodidad
df.rename(columns={"default.payment.next.month": "target"}, inplace=True)

---

In [40]:
# b) Tratamiento de categorías fuera de rango
df["EDUCATION"] = df["EDUCATION"].replace({0:4, 5:4, 6:4}).astype("category")
df["MARRIAGE"]  = df["MARRIAGE"].replace({0:3}).astype("category")
df["SEX"]       = df["SEX"].astype("category")

In [41]:
# c) Variables predictoras
X = df.drop(columns=["ID", "target"])
y = df["target"]

### Librerías

In [46]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'),
                                                  ['SEX', 'EDUCATION',
                                                   'MARRIAGE']),
                                                 ('num', 'passthrough',
                                                  ['LIMIT_BAL', 'AGE', 'PAY_0',
                                                   'PAY_2', 'PAY_3', 'PAY_4',
                                                   'PAY_5', 'PAY_6',
                                                   'BILL_AMT1', 'BILL_AMT2',
                                                   'BILL_AMT3', 'BILL_AMT4',
                                                   'BILL_AMT5', 'BILL_AMT6',
                                                   'PAY_AMT1', 'PAY_AMT2',
                                                   'PAY_AMT3', 'PAY_AMT4'...
                               feature_types=None, feature_weights=None,
                               gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.05,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=4, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=400, n_jobs=-1,
                               num_parallel_tree=None, ...))])

#### Ajuste de hiperparametros

In [48]:
# ------------- 6. Ajuste de hiperparámetros (opcional) ----
param_grid = {
    "model__n_estimators": [300, 500, 700],
    "model__max_depth":   [3, 4, 5],
    "model__learning_rate": [0.01, 0.05, 0.1],
    "model__subsample":   [0.8, 1.0],
    "model__colsample_bytree": [0.8, 1.0]
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grid = GridSearchCV(pipe, param_grid,
                    scoring="roc_auc",
                    cv=cv,
                    n_jobs=-1, verbose=2)

grid.fit(X_train, y_train)
print("Mejor AUC validación:", grid.best_score_)
print("Mejores hiperparámetros:", grid.best_params_)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Mejor AUC validación: 0.7844989606548571
Mejores hiperparámetros: {'model__colsample_bytree': 0.8, 'model__learning_rate': 0.01, 'model__max_depth': 5, 'model__n_estimators': 700, 'model__subsample': 0.8}


Test y métricas

In [49]:
best_model = grid.best_estimator_
# ------------- 7. Métricas con mejores hiperparámetros -------------------
best_pred  = best_model.predict(X_test)
best_proba = best_model.predict_proba(X_test)[:, 1]

print("\n=== Métricas con modelo optimizado ===")
print(classification_report(y_test, best_pred))
print("Accuracy:", accuracy_score(y_test, best_pred))
print("ROC AUC:", roc_auc_score(y_test, best_proba))
print("Confusion matrix:\n", confusion_matrix(y_test, best_pred))


=== Métricas con modelo optimizado ===
              precision    recall  f1-score   support

         0.0       0.88      0.81      0.84      4673
         1.0       0.48      0.62      0.54      1327

    accuracy                           0.77      6000
   macro avg       0.68      0.71      0.69      6000
weighted avg       0.79      0.77      0.78      6000

Accuracy: 0.7665
ROC AUC: 0.7795914608944164
Confusion matrix:
 [[3779  894]
 [ 507  820]]


## Despliegue en Google Collab

In [50]:
import os
from pyngrok import ngrok

# Configura tu token de ngrok
os.environ["NGROK_TOKEN"] = "AQUI_TU_TOKEN"
!ngrok authtoken $NGROK_TOKEN

# Inicia el servidor MLflow
mlflow_command = """
mlflow server \
  --backend-store-uri sqlite:///mlruns/tracking.db \
  --default-artifact-root ./mlruns \
  --host 0.0.0.0 \
  -p 5000 &
"""
get_ipython().system_raw(mlflow_command)

# Conecta ngrok al puerto 5000
public_url = ngrok.connect(5000, "http").public_url
print("MLflow UI disponible en:", public_url)


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
MLflow UI disponible en: https://7c1cb74371c3.ngrok-free.app


3. Entrenamiento y registro de tu modelo con mlflow

In [72]:
# Mejores hiperparámetros encontrados por GridSearchCV:
# {'model__colsample_bytree': 0.8, 'model__learning_rate': 0.01,
#  'model__max_depth': 5, 'model__n_estimators': 700,
#  'model__subsample': 0.8}

import mlflow
import mlflow.xgboost
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

# Configura la URI del servidor de MLflow
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("credit_card_default_prediction")

with mlflow.start_run(run_name="xgboost_final"):

    # Define el modelo con los mejores hiperparámetros
    model = XGBClassifier(
        colsample_bytree=0.8,
        learning_rate=0.01,
        max_depth=5,
        n_estimators=700,
        subsample=0.8,
        n_jobs=-1,
        enable_categorical=True,
        use_label_encoder=False,
        eval_metric='logloss'  # para evitar warnings
    )

    # Entrena el modelo
    model.fit(X_train, y_train)

    # Predicciones
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]

    # Registra métricas
    mlflow.log_metric("accuracy", accuracy_score(y_test, y_pred))
    mlflow.log_metric("roc_auc", roc_auc_score(y_test, y_proba))

    # Registra el modelo en MLflow
    mlflow.xgboost.log_model(model, artifact_path="model")

    # Guarda run_id
    run_id = mlflow.active_run().info.run_id
    print("Run guardado con ID:", run_id)


/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [03:42:47] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
2025/07/25 03:42:49 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:1028: UserWarning: [03:42:49] WARNING: /workspace/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  self.get_booster().save_model(fname)
2025/07/25 03:42:51 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Run guardado con ID: d26d9c2d684d4afdbcb6a2fb855dabe6
🏃 View run xgboost_final at: http://localhost:5000/#/experiments/1/runs/d26d9c2d684d4afdbcb6a2fb855dabe6
🧪 View experiment at: http://localhost:5000/#/experiments/1


5. Servir el modelo con MLflow como REST API

In [75]:
serve_cmd = """
mlflow models serve -m "models:/xgboost_credit_risk/1" -p 8001 --env-manager local &
"""
get_ipython().system_raw(serve_cmd)


 6. Enviar predicciones a la API vía /invocations

In [94]:
import requests
import json


X_sample = X_test[1:5]
print(X_sample)
input_list = X_sample.to_numpy().tolist()

# Formato para la API
data = {
    "inputs": input_list
}

# Envía POST request al modelo en producción
response = requests.post("http://localhost:8001/invocations", json=data)
print("Predicción:", response.text)


       LIMIT_BAL SEX EDUCATION MARRIAGE  AGE  PAY_0  PAY_2  PAY_3  PAY_4  \
24575   150000.0   1         1        1   31     -1     -1     -2     -2   
26766    50000.0   1         2        2   25      0      0      0      0   
2156    290000.0   2         1        2   25      0      0      0      0   
3179    500000.0   2         2        1   27     -2     -2     -2     -2   

       PAY_5  ...  BILL_AMT3  BILL_AMT4  BILL_AMT5  BILL_AMT6  PAY_AMT1  \
24575     -2  ...        0.0        0.0        0.0    11694.0       0.0   
26766      0  ...    49009.0    49949.0    50479.0    50702.0    1800.0   
2156       0  ...   296384.0   248801.0   241983.0   230925.0   15000.0   
3179      -2  ...    13570.0    10000.0    10000.0    10000.0    9983.0   

       PAY_AMT2  PAY_AMT3  PAY_AMT4  PAY_AMT5  PAY_AMT6  
24575       0.0       0.0       0.0   11694.0   30000.0  
26766    1844.0    2200.0    2000.0    1800.0    2038.0  
2156    10500.0   10000.0   15000.0    7844.0   23333.0  
3179    135

## Despligue en PC personal

In [ ]:
# -------------------- Cargar modelo guardado  ---------------------
import joblib

best_model = joblib.load("xgb_credit_default.pkl")
print("Modelo cargado desde disco: xgb_credit_default.pkl")

In [ ]:
import os
import mlflow
# --------------------   registro en MLflow --------------

mlflow.set_tracking_uri(f"file:///{os.path.abspath('mlruns')}")
mlflow.set_experiment("Entrega4_Deployment")

with mlflow.start_run() as run:
    mlflow.sklearn.log_model(best_model, artifact_path="model")
    mlflow.log_metric("roc_auc", roc_auc_score(y_test, best_proba))
    mlflow.log_metric("accuracy", accuracy_score(y_test, best_pred))

    # registro global
    mlflow.register_model(
        model_uri=f"runs:/{run.info.run_id}/model",
        name="XGBoost_Optimizado"
    )

    run_id = run.info.run_id
    print(f"\n Modelo registrado en run {run_id}")
    print("   Revisa la carpeta mlruns/0/" + run_id + " para ver el artefacto.")

In [ ]:
# --------------------  Generar payload de prueba ----------------------
import json

# Construir payload según el nuevo protocolo MLflow 2.x (dataframe_split)
payload = {
    "dataframe_split": {
        "columns": X_test.columns.tolist(),
        "data": X_test.values.tolist()
    }
}

# Guardar payload válido en payload.json
with open("payload.json", "w") as f:
    json.dump(payload, f, indent=2)

print("Payload de prueba guardado en payload.json:")
print(json.dumps(payload, indent=2))


### Código de despliegue

| Elemento                           | Descripción                                                                                |
| ---------------------------------- | ------------------------------------------------------------------------------------------ |
| **Archivo principal**              | `deployment_code.py` (contiene entrenamiento, registro y utilidades para servir el modelo) |
| **Rutas de acceso a los archivos** |                                                                                            |
|   • Dataset                        | `./data/UCI_Credit_Card.csv`                                                               |
|   • Carpeta de experimentos (runs) | `./mlruns/0/<run_id>/artifacts/model/`                                                     |
|   • Model Registry                 | `./mlruns/models/XGBoost_Optimizado/`                                                      |
|   • Payload de prueba              | `./payload.json`                                                                           |
| **Variables de entorno**           |                                                                                            |
|   • `MLFLOW_TRACKING_URI`          | `file:///…/scripts/deployment/mlruns`                                                      |
|   • `PORT`                         | `5001` (puerto del servidor de inferencia)                                                 |
|   • `NGROK_AUTHTOKEN`              | * token Ngrok*                                                               |

---

### Pasos para reproducir el despliegue

1. **Entrenar y registrar el modelo**


   Esto guarda el run y crea la versión `XGBoost_Optimizado/2` en el Model Registry.

2. **Servir el modelo localmente**

   ```bash
   mlflow models serve \
     --model-uri "models:/XGBoost_Optimizado/2" \
     --host 0.0.0.0 --port 5001 --workers 2 --no-conda
   ```

   El endpoint queda en `http://localhost:5001/invocations`.

3. **Probar la inferencia local**

   ```bash
   curl -X POST http://localhost:5001/invocations \
        -H "Content-Type: application/json" \
        -d @payload.json
   ```

4. **Exponer con Ngrok**

   ```bash
   ngrok authtoken <TU_TOKEN>
   ngrok http 5001
   # Salida típica:
   # Forwarding https://abcd1234.ngrok-free.app -> http://localhost:5001
   ```

5. **Inferencia remota (cualquier PC)**

   ```bash
   curl -X POST https://abcd1234.ngrok-free.app/invocations \
        -H "Content-Type: application/json" \
        -d @payload.json
   ```

   Devuelve, por ejemplo: `{ "predictions": [0] }`.

---

### Conceptos usados

* **Endpoint**: URL que expone una funcionalidad (aquí: `/invocations`).
* **Inferencia**: proceso de generar una predicción con el modelo.
* **Ngrok**: túnel HTTPS temporal que hace pública una URL interna.
* **Model Registry**: módulo de MLflow que versiona y gestiona modelos listos para producción.

Con estos pasos tu compañero puede reproducir el flujo completo y explicar el despliegue en el taller.


### Resultados al llamar el modelo de la terminal


   ```(deepdl) PS C:\Users\Lenovo\Desktop\MODULO METODOLOGÍAS AGILES\Proyecto\tdsp_template\scripts\deployment> curl.exe -X POST http://localhost:5001/invocations -H "Content-Type: application/json" -d "@payload.json"
{"predictions": [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0]}
   ```

### Captura de pantalla 

A continuación se muestra la captura de pantalla del resultado del modelo llamado desde la terminal

> ![Ciclo de vida TDSP](./Imagenes/predicciones_conda_power_shell.png)



---


**En el video final se presentará el funcionamiento de la inferencia remota**

---